# Figuring things out...

In [1]:
import configparser

In [29]:
config = configparser.RawConfigParser()   
config.read("../../api_keys.txt")

['../../api_keys.txt']

In [30]:
api_key = config.get("dark_sky", "api_key")

In [31]:
api_key = api_key + "/"

In [32]:
import requests

In [6]:
base_url = "https://api.darksky.net/forecast/"

In [45]:
exclude = "?exclude=[currently,minutely,hourly,alerts,flags]"

time:
Either be a UNIX time (that is, seconds since midnight GMT on 1 Jan 1970) or a string formatted as follows: [YYYY]-[MM]-[DD]T[HH]:[MM]:[SS][timezone]. timezone should either be omitted (to refer to local time for the location being requested), Z (referring to GMT time), or +[HH][MM] or -[HH][MM] for an offset from GMT in hours and minutes. The timezone is only used for determining the time of the request; the response will always be relative to the local time zone.

In [49]:
sample_time = ",2018-02-05T12:00:00"

In [50]:
r = requests.get(base_url+api_key+"37.8267,-122.4233"+sample_time+exclude)

In [67]:
j = r.json()

In [77]:
import json
from pandas.io.json import json_normalize

In [93]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], a)
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, str(i))
                i += 1
        else:
            out[name] = x

    flatten(y)
    return out

In [79]:
j

{'daily': {'data': [{'apparentTemperatureHigh': 66.27,
    'apparentTemperatureHighTime': 1517875200,
    'apparentTemperatureLow': 54.86,
    'apparentTemperatureLowTime': 1517918400,
    'apparentTemperatureMax': 66.27,
    'apparentTemperatureMaxTime': 1517875200,
    'apparentTemperatureMin': 53.53,
    'apparentTemperatureMinTime': 1517839200,
    'cloudCover': 0.18,
    'dewPoint': 50.32,
    'humidity': 0.76,
    'icon': 'partly-cloudy-day',
    'moonPhase': 0.69,
    'precipIntensity': 0,
    'precipIntensityMax': 0,
    'precipProbability': 0,
    'pressure': 1016.53,
    'summary': 'Partly cloudy until evening.',
    'sunriseTime': 1517843467,
    'sunsetTime': 1517881134,
    'temperatureHigh': 66.27,
    'temperatureHighTime': 1517875200,
    'temperatureLow': 54.86,
    'temperatureLowTime': 1517918400,
    'temperatureMax': 66.27,
    'temperatureMaxTime': 1517875200,
    'temperatureMin': 53.53,
    'temperatureMinTime': 1517839200,
    'time': 1517817600,
    'visibilit

In [97]:
flattened_j = flatten_json(j)

In [101]:
json_normalize(flattened_j).columns

Index(['apparentTemperatureHigh', 'apparentTemperatureHighTime',
       'apparentTemperatureLow', 'apparentTemperatureLowTime',
       'apparentTemperatureMax', 'apparentTemperatureMaxTime',
       'apparentTemperatureMin', 'apparentTemperatureMinTime', 'cloudCover',
       'dewPoint', 'humidity', 'icon', 'latitude', 'longitude', 'moonPhase',
       'offset', 'precipIntensity', 'precipIntensityMax', 'precipProbability',
       'pressure', 'summary', 'sunriseTime', 'sunsetTime', 'temperatureHigh',
       'temperatureHighTime', 'temperatureLow', 'temperatureLowTime',
       'temperatureMax', 'temperatureMaxTime', 'temperatureMin',
       'temperatureMinTime', 'time', 'timezone', 'visibility', 'windBearing',
       'windSpeed'],
      dtype='object')

In [37]:
import pandas as pd

# Iterate to find daily Penn Station for time range?

In [102]:
import datetime

In [108]:
datetime.datetime.today().strftime("%Y-%m-%d")

'2018-03-22'

## Make range of time for requests

In [119]:
date_range = [datetime.datetime.strptime("2018-02-05", "%Y-%m-%d") + datetime.timedelta(days=x) for x in range(0, 28)]

In [120]:
date_range = [date.strftime("%Y-%m-%d") for date in date_range]

In [127]:
date_range[0], date_range[len(date_range) - 1]

('2018-02-05', '2018-03-04')

In [123]:
sample_time_format = ",{date}T12:00:00"

In [128]:
for date in date_range:
    print(sample_time_format.format(date=date))

,2018-02-05T12:00:00
,2018-02-06T12:00:00
,2018-02-07T12:00:00
,2018-02-08T12:00:00
,2018-02-09T12:00:00
,2018-02-10T12:00:00
,2018-02-11T12:00:00
,2018-02-12T12:00:00
,2018-02-13T12:00:00
,2018-02-14T12:00:00
,2018-02-15T12:00:00
,2018-02-16T12:00:00
,2018-02-17T12:00:00
,2018-02-18T12:00:00
,2018-02-19T12:00:00
,2018-02-20T12:00:00
,2018-02-21T12:00:00
,2018-02-22T12:00:00
,2018-02-23T12:00:00
,2018-02-24T12:00:00
,2018-02-25T12:00:00
,2018-02-26T12:00:00
,2018-02-27T12:00:00
,2018-02-28T12:00:00
,2018-03-01T12:00:00
,2018-03-02T12:00:00
,2018-03-03T12:00:00
,2018-03-04T12:00:00


In [130]:
penn_station_lat_long = "40.750046,-73.992358"

In [131]:
penn_station_requests = []

for date in date_range:
    date_for_url = sample_time_format.format(date=date)
    r = requests.get(base_url+api_key+penn_station_lat_long+date_for_url+exclude)
    penn_station_requests.append(r)

In [133]:
penn_station_flattened = [flatten_json(r.json()) for r in penn_station_requests]

In [142]:
penn_station_daily_weather_df = pd.DataFrame(penn_station_flattened, index = pd.DatetimeIndex(date_range))

In [145]:
# penn_station_daily_weather_df.to_csv("penn_station_daily_weather_0205-0304.csv")